In [11]:
#kkep imports here
import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pcdet.ops.voxel import Voxelization
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.utils import common_utils
from tools.cfgs import dataset_configs
from tools.cfgs import kitti_models

In [9]:
dist_train = False
output_dir = cfg.ROOT_DIR
log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)
merge_all_iters_to_one_epoch = False
workers = 8
epochs = 50

In [15]:
from pcdet.config import cfg, cfg_from_yaml_file
from eval_utils.eval_utils import load_data_to_gpu
import pickle
cfg = cfg_from_yaml_file("cfgs/kitti_models/spg.yaml", cfg)
data_config = cfg.DATA_CONFIG

In [17]:
train_set, train_loader, train_sampler = build_dataloader(
        dataset_cfg=data_config,
        class_names=cfg.CLASS_NAMES,
        batch_size=4,
        dist=dist_train, workers=workers,
        logger=logger,
        training=True,
        merge_all_iters_to_one_epoch=merge_all_iters_to_one_epoch,
        total_epochs=epochs
    )

2021-11-25 15:41:42,591   INFO  Database filter by min points Car: 14357 => 13532
2021-11-25 15:41:42,591   INFO  Database filter by min points Car: 14357 => 13532
2021-11-25 15:41:42,592   INFO  Database filter by min points Pedestrian: 2207 => 2168
2021-11-25 15:41:42,592   INFO  Database filter by min points Pedestrian: 2207 => 2168
2021-11-25 15:41:42,593   INFO  Database filter by min points Cyclist: 734 => 705
2021-11-25 15:41:42,593   INFO  Database filter by min points Cyclist: 734 => 705
2021-11-25 15:41:42,605   INFO  Database filter by difficulty Car: 13532 => 10759
2021-11-25 15:41:42,605   INFO  Database filter by difficulty Car: 13532 => 10759
2021-11-25 15:41:42,608   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2021-11-25 15:41:42,608   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2021-11-25 15:41:42,609   INFO  Database filter by difficulty Cyclist: 705 => 581
2021-11-25 15:41:42,609   INFO  Database filter by difficulty Cyclist: 705 => 

In [ ]:
train_set[0]["gt_classification"] = np.zeros(train_set["batch_size"], train_set["voxel_num_points"][0])


In [41]:
train_set[0].keys()

dict_keys(['frame_id', 'gt_boxes', 'points', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'image_shape'])

In [45]:
train_set[0]["voxels"].shape

(4052, 32, 4)

In [44]:
train_set[0]["voxels"][0][0]

array([ 4.7128134,  7.1744437, -0.0765012,  0.27     ], dtype=float32)

In [51]:
train_set[0]["gt_boxes"].shape

(31, 8)

In [52]:
train_set[0]["gt_boxes"][0]

array([ 8.365651  ,  3.2791002 , -0.65901995,  1.2079192 ,  0.48316765,
        1.9024727 , -0.9977898 ,  2.        ], dtype=float32)

In [50]:
train_set[0]["points"][0]


array([ 14.355981 , -11.07246  ,  -0.8205786,   0.41     ], dtype=float32)